In [91]:
import json
import requests

def get_movies_from_tastedive(movie):
    params = {"q": movie, "type": "movies", "limit": 5}
    response = requests.get("https://tastedive.com/api/similar", params=params)
    return response.json()

In [92]:
def extract_movie_titles(movies):
    titles = []
    for result in movies["Similar"]["Results"]:
        titles.append(result["Name"])
    return titles

In [93]:
def get_related_titles(movies):
    all_related_titles = []
    for movie in movies:
        related_movies = get_movies_from_tastedive(movie)
        related_titles = extract_movie_titles(related_movies)
        for related_title in related_titles:
            if related_title not in all_related_titles:
                all_related_titles.append(related_title)
    return all_related_titles

In [100]:
def get_omdb_api_key():
    with open("omdb_api_key.json", "r") as inf:
        payload = json.load(inf)
    return payload

In [101]:
def get_movie_data(title):
    url = "http://www.omdbapi.com/"
    params = {"r": "json", "t": title}
    params = {**params, **get_omdb_api_key()}
    response = requests.get(url, params=params)
    return response.json()

In [102]:
def get_movie_rating(movie_data):
    ratings = movie_data["Ratings"]
    for rating in ratings:
        if rating["Source"] == "Rotten Tomatoes":
            value = rating["Value"]
            if value[-1] == "%":
                value = value[:-1]
            return int(value)
    return 0

In [103]:
def get_sorted_recommendations(movies):
    recommendations = []
    related_titles = get_related_titles(movies)
    for title in related_titles:
        data = get_movie_data(title)
        rating = get_movie_rating(data)
        recommendations.append((title, rating))
    recommendations.sort(key=lambda t: t[1], reverse=True)
    return [recommendation[0] for recommendation in recommendations]

In [104]:
get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

['Iron Man',
 'Date Night',
 'The Heat',
 'Baby Mama',
 'The Five-Year Engagement',
 'Sherlock Holmes: A Game Of Shadows',
 'Bachelorette',
 'Prince Of Persia: The Sands Of Time',
 'Pirates Of The Caribbean: On Stranger Tides',
 'The Tourist']